In [1]:
import os
import concurrent.futures
import multiprocess as mp
import random


from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
import lightkurve as lk
from astroquery.simbad import Simbad
import pandas as pd 
from numpy import genfromtxt


#tf
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [2]:
hdul = fits.open('/Users/michaelsong/Documents/GitHub/starzam/Labels/J_ApJS_236_42_table2.dat.gz.fits')

hdul.info()
print(hdul[1].data)

Filename: /Users/michaelsong/Documents/GitHub/starzam/Labels/J_ApJS_236_42_table2.dat.gz.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     114   ()      
  1  table2.dat    1 TableHDU       162   16094R x 20C   [I8, I4, I3, F5.3, F5.3, F5.2, F4.2, F4.2, F4.2, F5.2, F4.2, F4.2, F4.2, F5.2, F4.2, F4.2, F4.2, F5.2, F5.2, I1]   


[(757137, 4751, 139, 2.383, 0.012, -0.08, 0.3, 1.7, 0.14, 13.91, 0.41, 1.55, 0.13, 13.26, 0.39, 1.72, 0.14, 13.97, 0.41, 1)
 (892010, 4834, 151, 2.161, 0.023, 0.21, 0.3, 1.42, 0.29, 16.37, 1.38, 1.36, 0.28, 16.02, 1.34, 1.43, 0.3, 16.47, 1.39, 0)
 (892738, 4534, 135, 1.769, 0.022, -0.25, 0.3, 1.15, 0.21, 23.15, 1.62, 0.99, 0.18, 21.49, 1.48, 1.12, 0.2, 22.86, 1.6, 0)
 ...
 (12885196, 4846, 142, 2.878, 0.008, 0.07, 0.3, 1.55, 0.1, 7.5, 0.17, 1.46, 0.09, 7.29, 0.16, 1.57, 0.1, 7.55, 0.17, 1)
 (12885373, 4855, 144, 2.627, 0.009, 0.12, 0.3, 1.49, 0.09, 9.83, 0.22, 1.42, 0.09, 9.59, 0.22, 1.51, 0.1, 9.9, 0.22, 1)
 (12934574, 4834, 143, 2.387, 0.012, 0.01, 0.3, 1.02, 0.1, 10.72, 0.42, 0.99, 0.1, 10.57, 0.41, 1.03, 0.1, 10.74, 0.42, 2)]


In [3]:
test_lk = lk.search_lightcurve('KIC '+str(757137), author='Kepler',exptime=1800)[2].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
print(len(test_lk.time.value))
l = random.randrange(250, len(test_lk.time.value))
s = random.randrange(0,len(test_lk.time.value)-l)
print(s,l)
test_lk2 = test_lk.truncate(after=test_lk.time.value[s+l],before=test_lk.time.value[s])
test_periodogram = test_lk2.to_periodogram(frequency = [x for x in np.arange(1,24.5,0.1)])
freq = np.asarray([test_periodogram.frequency,test_periodogram.power*100000])
print(freq[1])

4134
769 3097
[3.98728835 5.23978766 3.68869096 4.26263808 2.39676612 2.63241334
 7.09936687 2.27450985 1.13083077 4.00145725 2.26164076 1.75136105
 9.29602755 6.38219462 2.19580687 5.10864266 2.70333227 1.78810054
 0.60297275 6.63340498 1.86488079 2.6196339  1.60743776 1.19494879
 1.12927478 3.93050777 0.40013264 1.84550159 1.92489994 2.90028191
 0.6349395  0.87037423 1.46262056 0.76590084 1.15000402 0.82965415
 1.32258456 1.57456313 0.55137049 1.04314215 1.80492838 0.70324002
 1.47628738 0.73767742 0.54324813 0.8008493  1.34266118 0.40496329
 0.56245915 0.8090544  0.1654346  0.58757217 0.77800906 0.25696758
 0.12888118 0.16779005 0.51350812 0.78961495 0.38030405 0.77289323
 0.37053317 0.30118936 0.96312232 0.47550952 0.72338277 0.25005494
 0.15095896 0.29026665 0.30534708 0.79899413 0.37289919 0.39538115
 0.10690973 0.49031415 0.36215044 0.28287058 0.15181652 0.59888814
 0.18415458 0.25812709 0.37160346 0.14938041 0.11295613 0.55495831
 0.2913743  0.25450717 0.26248563 0.49396225 0.2

In [4]:
def prepsr(i,sr,j):
    r = True
    if r:
        test_lk = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
        l = random.randrange(240, len(test_lk.time.value))
        s = random.randrange(0,len(test_lk.time.value)-l)
        lightcurve = test_lk.truncate(after=test_lk.time.value[s+l],before=test_lk.time.value[s])
    else:
        lightcurve = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
    
    periodogram = lightcurve.to_periodogram(frequency = [x for x in np.arange(1,24.5,0.1)])
    # periodogram = periodogram.bin(binsize=10, method='mean')
    freq = np.asarray([periodogram.frequency,periodogram.power*100000])
    # pd.DataFrame(freq[1]).to_csv("/content/drive/MyDrive/Research - Yihong Song and Rohit Prasanna/Yihong/Data/"+str(labeled_data["Kepler_ID"][i])+"_"+str(j)+".csv")
    return np.append(np.append(np.append(np.append(hdul[1].data[i][0:2],hdul[1].data[i][3]),hdul[1].data[i][5]),[j]),freq[1])

def getlc(i):
    lc_data = []
    sr = lk.search_lightcurve('KIC '+str(hdul[1].data[i][0]), author='Kepler',exptime=1800)
    # return prepsr(i,sr,1)
    for j in range(len(sr)):
        # x = wrap(prepsr(i,sr,j))
        # if x is not None:
        lc_data.append(prepsr(i,sr,j))
        # y = wrap(external_function_raises)
        # if y is not None:
        #     lc_data.append(y)
    return lc_data

def getdata(start):
    # for k in range(start,int(191449/100)+1,1):
    for k in range(start,start+1,1):
        lc_data = []
        for i in range(k*10,(k+1)*10,1):
            try:
                lc_data = lc_data+getlc(i)
            except:
                continue
            print("\r"+str(i),end="")
        # print(lc_data)
        pd.DataFrame(lc_data).to_csv("/Users/michaelsong/Documents/GitHub/starzam/Data_v5/"+str(1610+k)+".csv")

In [16]:
getdata(0)

9

In [11]:
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     print(executor.map(getdata, [1,2,3]))

with mp.Pool(8) as pool:
    print(pool.map(getdata, [x for x in range(1600,1610)]))

16065

16089[None, None, None, None, None, None, None, None, None, None]
